In [1]:
import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as plt

t_x = pd.read_csv("pa2_train_X.csv")
t_y = pd.read_csv("pa2_train_y.csv")
d_x = pd.read_csv("pa2_dev_X.csv")
d_y = pd.read_csv("pa2_dev_y.csv")

t_x.head()

,Gender,Age,Driving_License,Previously_Insured,Vehicle_Damage,Annual_Premium,Vintage,Region_Code_0,Region_Code_1,Region_Code_2,...,Policy_Sales_Channel_153,Policy_Sales_Channel_154,Policy_Sales_Channel_155,Policy_Sales_Channel_156,Policy_Sales_Channel_157,Policy_Sales_Channel_158,Policy_Sales_Channel_159,Policy_Sales_Channel_160,Policy_Sales_Channel_163,dummy
0,1,46,1,0,1,2630.0,215,0,0,0,...,0,0,0,1,0,0,0,0,0,1.0
1,0,25,1,0,0,35701.0,228,0,0,0,...,0,0,0,0,0,0,0,0,0,1.0
2,0,42,1,0,1,34300.0,239,0,0,0,...,0,0,0,0,0,0,0,0,0,1.0
3,1,23,1,1,0,35864.0,24,0,0,0,...,0,0,0,0,0,0,0,0,0,1.0
4,1,45,1,0,1,41773.0,267,0,0,0,...,0,0,0,0,0,0,0,0,0,1.0


In [2]:
num_features = ['Age', 'Annual_Premium', 'Vintage']

train_numf_max = t_x[num_features].max()
train_numf_min = t_x[num_features].min()

dev_numf_max = d_x[num_features].max()
dev_numf_min = d_x[num_features].min()

In [3]:
train_norm = []
for i in range(len(num_features)):
	norm_fea = [];
	cur_fea = num_features[i];
	range_val = train_numf_max[cur_fea] - train_numf_min[cur_fea];
	for j in range(len(t_x[cur_fea])):
		cur_val = t_x[cur_fea][j];
		norm = float((cur_val - train_numf_min[cur_fea]) / (range_val) )
		norm_fea.append(norm)
	train_norm.append(norm_fea)

train_norm = np.asarray(train_norm)
dict_train_norm = dict(zip(num_features, train_norm))

dev_norm = []
for i in range(len(num_features)):
	norm_fea = []
	cur_fea = num_features[i]
	range_val = dev_numf_max[cur_fea] - dev_numf_min[cur_fea]
	for j in range(len(d_x[cur_fea])):
		cur_val = d_x[cur_fea][j]
		norm = float((cur_val - dev_numf_min[cur_fea]) / (range_val) )
		norm_fea.append(norm)
	dev_norm.append(norm_fea)

dev_norm = np.asarray(dev_norm)
dict_dev_norm = dict(zip(num_features, dev_norm))

In [7]:
for i in range(len(num_features)):
  t_x[num_features[i]] = dict_train_norm[num_features[i]]
  d_x[num_features[i]] = dict_dev_norm[num_features[i]]


#move dummy column to first column
temp = t_x['dummy']
t_x = t_x.drop('dummy', 1) 
t_x.insert(0, 'dummy', temp)

temp = d_x['dummy']
d_x = d_x.drop('dummy', 1)
d_x.insert(0, 'dummy', temp)


In [8]:
def sigmoid(x):
    return np.piecewise(x, [x > 0], [lambda i: 1 / (1 + np.exp(-i)), lambda i: np.exp(i) / (1 + np.exp(i))])

def get_grad(X, y, w, lr, N):
	val = np.dot(X, w)
	y_pred = sigmoid(val)

	resd = y - y_pred
	grad = (lr * np.dot(X.T, resd)) / N
	return resd, grad

epsilon = 0.0002
def learning(X_val, y_val, lr, rp, max_itr):
	t = time.time()

	N = X_val.shape[0]
	d = X_val.shape[1]
	#w = np.random.normal(0.0, 1.0, size = d)
	w = np.zeros(d)
	X = np.asarray(X_val)
	y = np.asarray(y_val)
	y = y.flatten()

	itr_cnt = 0;
	grad_norm = 100;

	#while (grad_norm > epsilon) & (itr_cnt < max_itr):
	while itr_cnt < max_itr:
		pass
		itr_cnt += 1;
		resd, grad = get_grad(X, y, w, lr, N)
		w = w + grad
		for j in range(1, d):
			w[j] = np.sign(w[j]) * np.maximum( np.abs(w[j]) - lr*rp, 0)

		#w[1:] = np.sign(w[1:]) * np.maximum((np.linalg(w[1:], ord=1) - lr * rp), 0)
		grad_norm = np.linalg.norm(grad)
		
	elapsed = time.time() - t;
	return (w, elapsed, itr_cnt) 


In [9]:
def calculate_acc(X_val, y_val, weights):
	re = np.dot(np.asarray(X_val), weights)

	pred_y = [];
	for val in re:
	  if val >= 0:
	    pred_y.append(1)
	  else:
	    pred_y.append(0)

	true_y = np.asarray(y_val)

	cnt = 0;
	for i in range(len(pred_y)):
	  if true_y[i] == pred_y[i]:
	    cnt += 1;

	acc = (cnt / len(true_y)) * 100

	return acc

In [ ]:
reg_param = [10**0, 10**-1, 10**-2, 10**-3, 10**-4, 10**-5];

train_accs = [];
dev_accs = [];



for rp in reg_param:
	weights, elapsed, iteration = learning(t_x, t_y, 0.01, rp, 10000)
	print("Training with reg_param " + str(rp))
	print("Time took : " + str(elapsed) + " with iteration count : " + str(iteration) )

	
	train_acc = calculate_acc(t_x, t_y, weights)
	train_accs.append(train_acc)
	print("Accuracy of training data with reg_param " + str(rp) + " is : " + str(train_acc))

	dev_acc = calculate_acc(d_x, d_y, weights)
	dev_accs.append(dev_acc)
	print("Accuracy of validation data with reg_param " + str(rp) + " is : " + str(dev_acc))
	
	#weights = np.abs(weights)
	wg = dict(zip(list(t_x.columns), weights))
	wgo = dict(sorted(wg.items(), key = lambda x:x[1], reverse=True))
	wdf = pd.DataFrame(wg, index=[0])



print('train accuracy : ', train_accs)
print('validation accuracy : ', dev_accs)


t_acc = dict(zip(reg_param, train_accs))
d_acc = dict(zip(reg_param, dev_accs))

print(t_acc)
print(d_acc)